In [1]:
import pandas as pd
import numpy as np

In [ ]:
path = '/content/drive/MyDrive/hehe'

# User Data

In [ ]:
userData = pd.read_csv(f'{path}/Userfinal.csv')
# Bỏ cột date, vì về sau mình chỉ quan tâm tới account_id unique 
userData = userData.drop(columns='date')
userData

In [ ]:
# HÀM XỬ LÝ ĐỂ NHÓM CÁC ACCOUNT ID bị trùng 
# Lưu ý: Với tài khoản chơi game, thì level của tài khoản là sẽ tăng theo từng thời điểm, nên mình chỉ nên lấy max của level 
# còn các trường dữ liệu còn lại thì có thể nhóm mỗi id và lấy tổng trên tất cả các ngày 

mapFunctions = {}
columnNames = userData.columns.to_list()

# Viết hàm xử lý với từng cột (trường)
for column in columnNames:
    if column == 'account_id':
        continue   # Bỏ qua vì mình sẽ nhóm các cột còn lại theo cột account, nên không cần xử lý cột account 
    elif column == 'level':
        mapFunctions[column] = 'max'    # lấy hàm max như giải thích ở phía trên
    else:
        mapFunctions[column] = 'sum'

mapFunctions

In [ ]:
# Nhóm theo account_id để có được unique
uniqueData = userData.groupby('account_id').agg(mapFunctions)
uniqueData

# Cluster Data

In [ ]:
clusterData = pd.read_csv(f'{path}/df_Cluster.csv')
clusterData

In [ ]:
# Số lượng unique của account_id khác nhau (397582 != 399128) có thể là do lúc xử lý outlier, một số account_id được xem là outlier đã bị loại bỏ
# nên lúc map vào, ta cũng không cần xem xét các account_id đó 
clusterData['account_id'].nunique()

# Map

In [ ]:
mapData = uniqueData.merge(clusterData, how='left', on='account_id')
# Loại những account_id đã bị remove trong quá trình xử lý dữ liệu 
# nghĩa là những account_id này sẽ có cluster là NaN sau khi merge 
mapData = mapData.dropna(subset=['Clusters'])
# Đưa các cột về đúng kiểu định dạng vốn có (vì sau khi merge các cột của cluster bị đưa về kiểu float, trong khi kiểu ban đầu là itn)
mapData = mapData.convert_dtypes() 
mapData

In [ ]:
mapData.to_csv(f'{path}/mapData.csv', index=False)